# Client

Demo of a client interacting with a remote agent. 

In [3]:
from langserve import RemoteRunnable

remote_runnable = RemoteRunnable("http://localhost:8000/")

Remote runnable has the same interface as local runnables

In [4]:
await remote_runnable.ainvoke({"input": "hi!"})

{'output': 'Hello! How can I assist you today?'}

In [5]:
remote_runnable.invoke({"input": "what does eugene think of cats?"})

{'output': 'Eugene thinks that cats like fish.'}